In [1]:
import sys
sys.path.append("../")

In [2]:
# statement_name = "../data/Справка_о_движении_денежных_средств_0819.pdf"
# incomes = 9678606.76
# expences = -9128682.41
# bank_name = "tbank"

statement_name = "../data/ozonbank_document_5189822.pdf"
incomes = 79744.0
expences = -78949
bank_name = "ozon"



# 0. Настройка

In [3]:
from src.finman.utils.pdf import extract_text_from_pdf
from src.finman.expenses import tbank, ozon
from src.finman.utils.gsheet import GSheetWorker

In [4]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

gsw = GSheetWorker(gcreds_file)

In [5]:
if bank_name == "tbank":
    trans_col = "Сумма операции в валюте карты"
    date_col = "Дата операции"
    time_col = "Время операции"
    spreadsheet_name = "t-bank (main)"
    parse_operations = tbank.parse_operations

elif bank_name == "ozon":
    trans_col = "Сумма операции"
    date_col = "Дата операции"
    time_col = "Время операции"
    spreadsheet_name = "ozon-bank (main)"
    parse_operations = ozon.parse_operations

# 1. Прочитать выписку

In [6]:
text = extract_text_from_pdf(statement_name)
transactions_df = parse_operations(text)

In [7]:
transactions_df.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа
0,4348aedc63,2024-08-16,2024-08-16 18:21:38,-482.0,2125895187,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
1,7725e5161f,2024-08-13,2024-08-13 19:02:08,938.0,2107392458,"Возврат оплаты за товары/услуги, купленные на ..."
2,8e154900e2,2024-08-13,2024-08-13 09:24:59,-2682.0,2103792680,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
3,71275ad89c,2024-08-13,2024-08-13 09:24:56,2690.0,1280184534,Перевод A4226062454289120000000011310501 через...
4,ead5097e9a,2024-08-12,2024-08-12 10:29:24,-2871.0,2097210103,"Оплата товаров/услуг на Платформе Ozon, заказ ..."


In [8]:
assert transactions_df.loc[transactions_df[trans_col] > 0][trans_col].sum() == incomes
assert transactions_df.loc[transactions_df[trans_col] < 0][trans_col].sum() == expences

# 2. Запроцессить гугол

In [9]:
gs_df = gsw.get_df(sheet_id, spreadsheet_name)

In [10]:
gs_df.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа


In [14]:
missing_recrods = transactions_df.loc[~transactions_df['id'].isin(gs_df['id'])]

In [15]:
missing_recrods.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа
0,4348aedc63,2024-08-16,2024-08-16 18:21:38,-482.0,2125895187,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
1,7725e5161f,2024-08-13,2024-08-13 19:02:08,938.0,2107392458,"Возврат оплаты за товары/услуги, купленные на ..."
2,8e154900e2,2024-08-13,2024-08-13 09:24:59,-2682.0,2103792680,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
3,71275ad89c,2024-08-13,2024-08-13 09:24:56,2690.0,1280184534,Перевод A4226062454289120000000011310501 через...
4,ead5097e9a,2024-08-12,2024-08-12 10:29:24,-2871.0,2097210103,"Оплата товаров/услуг на Платформе Ozon, заказ ..."


In [16]:
missing_recrods[date_col] = missing_recrods[date_col].astype(str)
missing_recrods[time_col] = missing_recrods[time_col].dt.strftime('%Y-%m-%d %H:%M')

gsw.insert_df(missing_recrods, sheet_id, spreadsheet_name)